In [1]:
# pair the response 

In [ ]:
from datasets import load_dataset
dataset = load_dataset("McGill-NLP/FaithDial")
print(len(dataset['test']))

In [25]:
# Update with GT Response in files that didn't have it.. 
input_file = "T5_gen_WoW.txt" # #input_['knowledge'],input_['prompt'],resp

In [30]:
with open(input_file, 'r') as f:
    input_data = f.read().split("\n")

# pipe delimited knowledge, history and pred_response
input_data = [x.split("|") for x in input_data]

for idx, test_sample in enumerate(dataset['test']):
    knowledge, prompt, pred_resp = input_data[idx]
    original_resp = test_sample['original_response']
    gt_resp = test_sample['response']
    input_data[idx].extend([original_resp, gt_resp])

In [33]:
all_results = [[cell if cell is not None else '' for cell in row] for row in input_data]
data_dump = "\n".join(['|'.join(res) for res in all_results])

with open("T5_gen_WoW.txt", 'w') as f:
    f.write(data_dump)


In [ ]:
# Analysis

In [ ]:
# [input_['knowledge'], input_['prompt'], resp,
#                          test_sample['original_response'], test_sample['response']]

In [26]:
len(input_data)

3539

### Metrics

In [34]:
from datasets import load_metric
from tqdm import tqdm
from transformers import AutoTokenizer
from bert_score import score as bert_score

In [36]:
metric_rouge = load_metric("rouge")
metric_bleu = load_metric("bleu")

In [73]:
# input_file = "T5_gen_WoW.txt"
# input_file = "T5_edit_FaithDial_khorr.txt"
input_file = "T5_gen_FaithDial.txt"
with open(input_file, 'r') as f:
    input_data = f.read().split("\n")

# pipe delimited knowledge, history and pred_response
input_data = [x.split("|") for x in input_data]

In [74]:
print(input_data[0])
# Drop the Header
input_data = input_data[1:]

['knowledge', 'prompt', 'model_response', 'original_response', 'gt_response']


In [76]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [77]:
predictions = []
references = []

tok_predictions = []
tok_references = []

for fields in tqdm(input_data):
    _, _, pred_resp, _, gt_resp = fields
    tok_predictions.append(tokenizer.tokenize(pred_resp))
    tok_references.append([tokenizer.tokenize(gt_resp)])
    predictions.append(pred_resp)
    references.append([gt_resp])

    # print(len(knowledge), len(history), len(pred_response))

100%|██████████| 3539/3539 [00:00<00:00, 6051.65it/s]


In [78]:
len(predictions), len(references)

(3539, 3539)

In [79]:
print(predictions[:2])
print(references[:2])

["I'm not sure, but they're currently in New York City, New York, Los Angeles, and Miami Beach.", "I'm not sure, but they are currently in New York City, East Hampton, and Miami Beach."]
[["I don't know how good they are, but Dylan's Candy Bar has a chain of candy shops in various cities."], ["I don't know, really, but they also are a supplier of candy."]]


In [80]:
res_bleu = metric_bleu.compute(predictions=tok_predictions, references=tok_references)

In [81]:
res_rouge = metric_rouge.compute(predictions=predictions, references=references)

In [82]:
_, _, bert_score_f1 = bert_score(
    predictions, 
    references, 
    lang="en",
    model_type="distilbert-base-uncased",
    return_hash=False
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [83]:
print(f"BLEU: {res_bleu['bleu']:.3f}")
print(f"ROUGE-1, ROUGE-2, ROUGE-L: {res_rouge['rouge1'].mid.fmeasure:.3f}, {res_rouge['rouge2'].mid.fmeasure:.3f}, {res_rouge['rougeL'].mid.fmeasure:.3f}")
print(f"BERTScore: {bert_score_f1.mean().item():.3f}")

BLEU: 0.165
ROUGE-1, ROUGE-2, ROUGE-L: 0.405, 0.190, 0.351
BERTScore: 0.821


In [ ]:
# all_results = []
# for test_sample in tqdm(dataset['test'], total=len(dataset['test'])):
#     input_ = {'knowledge': test_sample['knowledge'],
#             'prompt': test_sample['history'][-1]}
    
#     resp = infer(model, max_input_length=MAX_INPUT_LENGTH, max_output_length=100, device=DEVICE, **input_)
#     all_results.append([input_['knowledge'], input_['prompt'], resp,
#                          test_sample['original_response'], test_sample['response']])

# # clean_all-results
# all_results = [[cell if cell is not None else '' for cell in row] for row in all_results]
# data_dump = "\n".join(['|'.join(res) for res in all_results])

# with open("T5_edit_FaithDial_khorr.txt", 'w') as f:
#     f.write(data_dump)

In [1]:
# most dissimilarity??

In [2]:
import pandas as pd


In [3]:
fre_df = pd.read_csv("T5_edit_FaithDial_khorr.txt", sep="|")
drg_df = pd.read_csv("T5_gen_FaithDial.txt", sep="|")

In [6]:
drg_df['model_response']

0       I'm not sure, but they're currently in New Yor...
1       I'm not sure, but they are currently in New Yo...
2          They have 7,000 candies from around the world.
3       I'm not sure, but it's owned by Dylan Lauren, ...
4       Ah, OK. Did you know ancient Greece advised at...
                              ...                        
3534    Yeah, and they play at the United Center, whic...
3535    I'm a bot, so I can't sing. I know it can be d...
3536    Sure, you can sing with or without accompanime...
3537    That's nice. Do you have an excellent singing ...
3538    I'm a bot, so I don't have favorites. I know t...
Name: model_response, Length: 3539, dtype: object

In [5]:
fre_df['model_response']

0       I don't have any preferences. I know that ther...
1            I don't know, but they are a candy supplier.
2       I don't know but it has 7000 candies from arou...
3                   I can't say but Dylan Lauren owns it.
4       I see, did you know that athletes in ancient G...
                              ...                        
3534    Yes, they played their home games at the Unite...
3535    I'm a bot and can't sing. I do know that it ca...
3536    Yeah, musical instruments can be accompanied b...
3537    I see, did you know that excellence in singing...
3538    I'm a bot, so I can't sing. What I know is tha...
Name: model_response, Length: 3539, dtype: object